<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/flexin_ca%24h__24_(x_pipswithpython).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hurst
!pip install yfinance
!pip install pandas_ta
!pip install requests
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=3ca17e22b910778fde61645eb361353456ebcf9b70c5f4e4d64c99500795d890
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


Functions

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from hurst import compute_Hc
import pandas_ta as ta
from IPython.display import clear_output
from time import sleep

In [3]:
def get(t, p, i):
  '''
    Get financial data
      param t: ticker
      param p: period
      param i: interval
  '''
  try:
    d = yf.Ticker(t).history(period=p, interval=i)
    d['Ticker'], d['Interval'] = t, i
    d.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    return d
  except:
    return f'Error collect data for {t}.'

In [4]:
def telegram(message):
  '''
    Send message to telegram
      param message: message
  '''
  token = '6183952601:AAFbfrKfZb3iKX6GfYZQnatxo0lg1S4lL8Y'
  id = '@flexinflexinbot1'
  url = f'https://api.telegram.org/bot{token}/sendMessage'
  try:
    response = requests.post(url, json={'chat_id': id, 'text': message})
    return response
  except:
    return f'Error sending message to telegram.'

Fix

In [5]:
def date(d):
  '''
    Fix datetime
      param d: data
  '''
  try:
    if 'Datetime' not in d.columns:
      d['Datetime'] = pd.to_datetime(d.index)
      return d
    else:
      return d
  except:
    return f'Error in fix datetime.'

In [6]:
def drop(d):
  '''
    Drop NaN rows
      param d: data
  '''
  try:
    d = d.dropna()
    return d
  except:
    return f'Error in drop NaN rows.'

In [7]:
def transition():
  '''
    Transition
  '''
  sleep(5)
  clear_output()

Technical

In [8]:
def fast(d):
  '''
    Compute fast exponential moving average
      param d: data
  '''
  try:
    d['Fast'] = d['Close'].ewm(50).mean()
    return d
  except:
    return f'Error compute fast exponential moving average.'

In [9]:
def twentyfour(d):
  '''
    Compute shifted fast exponential moving average
      param d: data
  '''
  try:
    d['Twentyfour'] = d['Fast'].shift(24)
    return d
  except:
    return f'Error compute shifted fast exponential moving average.'

In [10]:
def slow(d):
  '''
    Compute slow exponential moving average
      param d: data
  '''
  try:
    d['Slow'] = d['Close'].ewm(100).mean()
    return d
  except:
    return f'Error compute slow exponential moving average.'

In [11]:
def f(d):
  '''
    Compute flexin exponential moving average
      param d: data
  '''
  try:
    d['Flexin'] = d['Close'].ewm(len(d['Close'])/2).mean()
    return d
  except:
    return f'Error compute flexin exponential moving average.'

Quantitative

In [12]:
def log(d):
  '''
    Compute log return
      param d: data
  '''
  try:
    d['Log'] = np.log(d['Close']/d['Close'].shift(1))
    return d
  except:
    return f'Error compute log return.'

In [13]:
def hurst(d):
  '''
    Compute hurst exponent
      param d: data
  '''
  try:
    H = lambda x: compute_Hc(x)[0]
    d['Hurst'] = d['Log'].rolling(window=100).apply(H)
    return d
  except:
    return f'Error compute hurst exponent.'

In [14]:
def smooth(d):
  '''
    Compute smooth hurst exponent of log returns
      param d: data
  '''
  s = [5, 25, 50, 75, 100]
  try:
    for smooth in s:
      d[f'H_{smooth}'] = d['Hurst'].ewm(smooth).mean()
      continue
    return d
  except:
    return f'Error compute smooth hurst exponent.'

In [15]:
def adx(d):
  '''
    Compute average directional index
      param d: data
      param l: length
  '''
  import pandas_ta as ta
  try:
    adx = ta.adx(d['High'], d['Low'], d['Close'], length=14)
    d = d.join(adx)
    return d
  except:
    return f'Error compute average directional index.'


Signals

In [16]:
def long(d):
  '''
    Long
      param d: data
  '''
  try:
    ticker = d['Ticker'][-1]
    date = d['Datetime'][-1]
    interval = d['Interval'][-1]
    telegram(f'📲 {date} | Buy - {ticker}, {interval} (ig: @pipswithpython).')
    return f'📲 {date} | Buy - {ticker}, {interval} (ig: @pipswithpython).'
  except:
    return f'Error long position.'

In [17]:
def short(d):
  '''
    Short
      param d: data
  '''
  try:
    ticker = d['Ticker'][-1]
    date = d['Datetime'][-1]
    interval = d['Interval'][-1]
    telegram(f'📲 {date} | Sell - {ticker}, {interval} (ig: @pipswithpython).')
    return f'📲 {date} | Sell - {ticker}, {interval} (ig: @pipswithpython).'
  except:
    return f'Error short position.'

In [18]:
def nothin(d):
  '''
    No signal
      param d: data
  '''
  try:
    ticker = d['Ticker'][-1]
    date = d['Datetime'][-1]
    interval = d['Interval'][-1]
    return f'📲 Nothin for {ticker}, {date}, {interval} (ig: @pipswithpython).'
  except:
    return f'Error in no signal function.'

Verify

In [26]:
def verify(d):
  '''
    Verify
      param d: data
  '''
  v = None
  try:
    if d['Hurst'][-1]>0.55 and d['H_5'][-1]>d['H_25'][-1] and d['H_25'][-1]>d['H_50'][-1] and d['H_50'][-1]>d['H_75'][-1] and d['H_75'][-1]>d['H_100'][-1]:
      v = True
    else:
      v = False
    return v
  except:
    return f'Error in verify function.'

In [33]:
def position(d):
  '''
    Position define
      param d: data
  '''
  try:
    if d['Hurst'][-1]>0.55 and d['Fast'][-1]>d['Slow'][-1] and d['ADX_14'][-1]>25 and d['DMP_14'][-1]>d['DMN_14'][-1] and d['Twentyfour'][-1]>d['Flexin'][-1]:
      long(d)
    elif d['Hurst'][-1]>0.55 and d['Fast'][-1]<d['Slow'][-1] and d['ADX_14'][-1]<25 and d['DMP_14'][-1]<d['DMN_14'][-1] and d['Twentyfour'][-1]<d['Flexin'][-1]:
      short(d)
    else:
      nothin(d)
  except:
    return f'Error in position function.'

In [34]:
def signal(d):
  '''
    Signals
      param d: data
  '''
  if verify(d) == True:
    print(position(d))
  else:
    print(nothin(d))


App

In [35]:
def apply(t, p, i):
  '''
    Apply all functions
      param t: ticker
      param p: period
      param i: interval
  '''
  try:
    d = get(t, p, i)
    d = date(d)
    d = log(d)
    d = hurst(d)
    d = smooth(d)
    d = fast(d)
    d = twentyfour(d)
    d = slow(d)
    d = f(d)
    d = adx(d)
    d = drop(d)
    return d
  except:
    return f'Error in apply all functions.'

In [36]:
def flexin(t, p, i):
  '''
    Flexin
      param t: ticker
      param p: period
      param i: interval
  '''
  f = apply(t, p, i)
  signal(f)

In [37]:
wallet = ['^HSI',
          '^N225',
          '^DJI',
          'BTC-USD']

In [ ]:
while True:
  for w in wallet:
    flexin(w, '5d', '1m')
    transition()